In [1]:
import pulp
import numpy as np
import pandas as pd
import time

start_time = time.time()
EBB = pd.read_csv("portfolio_data.csv", sep = ',', encoding = "ISO-8859-1")

We import the relevant packages and data here. The start_time variable is introduced in order to measure the amount of time that the program runs

In [2]:
# Pool Parameters - these can be changed depending on the relevant portfolio criteria per deal
max_ADB =  400000000   #Target Amount of the portfolio
max_RV = max_ADB*0.55 #RV concentration
ReplenishmentGap = 10000 #maximum "lapda" allowed

#Client Concentration
Top1_5 = max_ADB*0.020 #five largest clients can't be larger than 2% of the portfolio, etc.
Top6_10 = max_ADB*0.015
Top11_15 = max_ADB*0.010
Top16_30 = max_ADB*0.0075
TopRest = max_ADB*0.005 #all clients outside the Top30 should not be larger than 0.50%
TopXCriteria = [Top1_5, Top1_5, Top1_5, Top1_5, Top1_5, Top6_10, Top6_10, 
                Top6_10, Top6_10, Top6_10, Top11_15, Top11_15, Top11_15, 
                Top11_15,Top11_15, Top16_30, Top16_30, Top16_30,Top16_30,
                Top16_30,Top16_30,Top16_30,Top16_30,Top16_30,Top16_30,
                Top16_30,Top16_30,Top16_30,Top16_30,Top16_30]
#Portfolio Concentration
TopSector = max_ADB*0.235
max_SME = max_ADB*0.35
max_Retail = max_ADB*0.02
max_remaining_term = max_ADB*0.01
max_non_monthly = max_ADB*0.05


#Solving Parameters
calc_time = 100 #maximum solving time in seconds
max_iter = 10 #maximum number of iterations
solver_Gurobi = pulp.GUROBI_CMD(timeLimit=calc_time, logPath='stats.log')#Gurobi, commercial solver
solver_CBC = pulp.PULP_CBC_CMD(timeLimit=calc_time,  logPath='stats.log') #default PulP solver (open source)
solver = solver_CBC #variable that allows you to choose your solver
InitialPoolCut = True  #set to False when entering the topping-up phase

This codeblock sets the relevant parameters for the problem. First the relevant parameters for the capacity constraints are set. Subsequently the parameters for the solver are created. We can make a decision on which solver to use here (Gurobi/CBC) and whether we want to solve the initial selection problem or the topping-up problem. Furthermore, we can set the maximum solving time and maximum number of iterations during the lazy constraint evaluation process. A LogPath was provided to the solver in order for us to be able to debug solving performance by inspecting the log.

In [3]:
# Create some variables that are needed to set-up the problem
DiscountedBalance = EBB['agg_current_balance']
PreviousPool = EBB['PreviousPool']*DiscountedBalance

# set linear optimization problem
LO_problem = pulp.LpProblem("BumperKnapSack", pulp.LpMaximize)

# create decision variables
n = len(DiscountedBalance) 
x = [pulp.LpVariable(f'{i}', cat='Binary') for i in range(n)]

LO_problem += (1-InitialPoolCut)*pulp.lpDot(x , PreviousPool)+ 
                InitialPoolCut*pulp.lpDot(x , DiscountedBalance),"ObjFunc" 

We create some useful general variables and decision variables here. The variables "x" represents the decision variable. x is a binary variable which is set to 1 if a particular lease contract is selected. DiscountedBalance represents translates to the "revenue" in a general knapsack setting. The PreviousPool variable represents lease contracts that were selected in the portfolio of previous month. The last line of code represents the objective function which is a combination of equation 19/20 in our paper, depending on which subproblem we are solving (initial selction or topping-up phase).

In [4]:
# Constraints
solution_ADB = pulp.lpDot(x , DiscountedBalance)
lapda = max_ADB - solution_ADB
LO_problem += ReplenishmentGap >= lapda #lapda should be lower than the threshold but can't be negative
LO_problem += pulp.lpDot(x, EBB['PV_CURRENT_MONTH_RV']) <= max_RV  #cap on Residual Value%
LO_problem += pulp.lpDot(x, DiscountedBalance*(EBB['CORP_GOVT_RETAIL_DESC']=='SME')) <= max_SME
LO_problem += pulp.lpDot(x, DiscountedBalance*(EBB['CORP_GOVT_RETAIL_DESC']=='Retail')) <= max_Retail
LO_problem += pulp.lpDot(x, DiscountedBalance*(EBB['REMAINING_DURATION']>60)) <= max_remaining_term
LO_problem += pulp.lpDot(x, DiscountedBalance*(EBB['PAYMENT_FREQ_NO']!='Monthly')) <= max_non_monthly

sector = EBB['NACE_Aggregated_Group_Desc'].unique().tolist() #limit exposure to a certain industry sector
sector.remove('L - REAL ESTATE ACTIVITIES')
for S in sector:
    LO_problem += pulp.lpDot(x, DiscountedBalance*(EBB['NACE_Aggregated_Group_Desc']==S)) <= TopSector

The relevant constraints are added to the model here (expect for the client concentration constraints, which are handled below). Lapda is defined as the difference between the Target Amount and the actual solution.

In [5]:
#Concentration Criteria are added for the largest clients only
TotalConstraints = pd.DataFrame(columns = ["SECURITISATION_GROUP_NO", "TopX"])
EBB_Grouped = EBB.groupby(['SECURITISATION_GROUP_NO'], as_index=False).sum() #group the available leases by customer
large_clients = (EBB_Grouped['agg_current_balance']>Top1_5).sum()
EBB_PreviousSol = EBB_Grouped.sort_values(['agg_current_balance'], 
                                          ascending=False, ignore_index=True)   
EBB_PreviousSol['TopX'] = np.NaN
EBB_PreviousSol['TopX'] = EBB_PreviousSol['TopX'].fillna(Top1_5)
AdditionalConstraints = EBB_PreviousSol.loc[0:large_clients, 
                                            ['SECURITISATION_GROUP_NO', 'TopX']]
for i,j in list(zip(AdditionalConstraints['SECURITISATION_GROUP_NO'],
                    AdditionalConstraints['TopX'])):
        LO_problem += pulp.lpDot(x, DiscountedBalance*(EBB['SECURITISATION_GROUP_NO']==i))<= j
TotalConstraints = TotalConstraints.append(AdditionalConstraints)

No client group (identified by SECURITISATION_GROUP_NO in the data) can represent more than 2% of the portfolio, constraints are added to reflect that. Only for clients which have a portfolio that is actually bigger than 2% of the portfolio, constraints are added to the model, so only a modest amount of constraints are added in this step.

In [6]:
if InitialPoolCut == False: #these criteria are only applicable during the topping-up phase
    #based on the ranking before topping-up, concentration criteria are assigned (to minimize buybacks)
    EBB_Grouped = EBB.groupby(['SECURITISATION_GROUP_NO'], as_index=False).sum() #grouping EBB
    large_clients = (EBB_Grouped['agg_current_balance']>Top1_5).sum()
    EBB_PreviousSol = EBB[EBB['PreviousPool']==1]
    EBB_PreviousSol = EBB_PreviousSol.groupby(['SECURITISATION_GROUP_NO'], 
                                              as_index=False).sum()
    EBB_PreviousSol = EBB_PreviousSol.sort_values(['agg_current_balance'], 
                                                  ascending=False, ignore_index=True)   
    EBB_PreviousSol['TopX'] = pd.Series(TopXCriteria)
    EBB_PreviousSol['TopX'] = EBB_PreviousSol['TopX'].fillna(TopRest)
    AdditionalConstraints = EBB_PreviousSol.loc[0:large_clients, 
                                                ['SECURITISATION_GROUP_NO', 'TopX']]   
    for i,j in list(zip(AdditionalConstraints['SECURITISATION_GROUP_NO'],
                        AdditionalConstraints['TopX'])):
        LO_problem += pulp.lpDot(x, DiscountedBalance*(EBB['SECURITISATION_GROUP_NO']==i))<= j
    TotalConstraints = TotalConstraints.append(AdditionalConstraints)

During the topping-up phase, it is important that the client concentration criteria are ranked in accordance with the solution of the problem in the previous period. If you don't do this, you might end up with a large client for which a smaller concentration criteria suddenly is applicable, which would initiate substantial substitutions

In [7]:
#The initial LO problem is solved
LO_problem.solve(solver)
print("Status:", pulp.LpStatus[LO_problem.status])
if pulp.LpStatus[LO_problem.status]=='Not Solved':
    print('problem was not solved')
else:
    print("Lapda = ", max_ADB - pulp.value(pulp.lpDot(x , DiscountedBalance))) #value of lapda

Status: Optimal
Lapda =  3.7599980235099792


The problem is solved for the first time.

In [8]:
def retrieve_sol(): #function to retrieve solution
    df = []
    for v in LO_problem.variables():
        if "__dummy" not in v.name:
            df.append((v.name, v.varValue))
    df = pd.DataFrame(df, columns=['a', 'b'], dtype=np.int32)
    df['a']=df['a'].astype(str).astype(int)
    df = df.sort_values(['a'])
    df = df.reset_index(drop=True)
    return df

A function is written to retrieve the solution from the relevant PulP variables

In [9]:
df = retrieve_sol()
frames = [EBB, df]
EBB_sol = pd.concat(frames, axis=1)
EBB_sol.to_csv('initial_sol.csv', index = False, header=True)

def CheckBreaches(): #function to check for breaches
    EBB_sol = pd.concat(frames, axis=1)
    EBB_solgroup = EBB_sol[EBB_sol['b']==1]
    EBB_solgroup = EBB_solgroup.groupby(['SECURITISATION_GROUP_NO'], 
                                        as_index=False).sum()
    EBB_solgroup = EBB_solgroup.sort_values(['agg_current_balance'], 
                                            ascending=False, ignore_index=True)   
    EBB_solgroup['TopX'] = pd.Series(TopXCriteria)
    EBB_solgroup['TopX'] = EBB_solgroup['TopX'].fillna(TopRest)
    EBB_solgroup['Breach'] = EBB_solgroup['agg_current_balance']
                                            >EBB_solgroup['TopX']   
    AdditionalConstraints = EBB_solgroup.loc[EBB_solgroup['Breach']==True, 
                                        ['SECURITISATION_GROUP_NO', 'TopX']]
    return AdditionalConstraints

A function is written that checks for breaches of any client concentration constraints.

In [10]:
no_iter = 0
while no_iter < max_iter:
    no_iter += 1
    df = retrieve_sol()
    frames = [EBB, df]
    AdditionalConstraints = CheckBreaches()
    if AdditionalConstraints['TopX'].count()==0:
        print('total number of solving iterations needed:', no_iter-1)
        if InitialPoolCut == False:
            print("ADB of contracts repurchased = ",
                  PreviousPool.sum() - pulp.value(pulp.lpDot(x , PreviousPool)))
        break        
    for i,j in list(zip(AdditionalConstraints['SECURITISATION_GROUP_NO'],
                        AdditionalConstraints['TopX'])):
        LO_problem += pulp.lpDot(x, DiscountedBalance*(EBB['SECURITISATION_GROUP_NO']==i))<= j
    TotalConstraints = TotalConstraints.append(AdditionalConstraints)
    LO_problem.solve(solver)
    print("Status:", pulp.LpStatus[LO_problem.status])
    if pulp.LpStatus[LO_problem.status]=='Not Solved':
        print('problem was not solved')
    else:
        print("Lapda = ", max_ADB - pulp.value(pulp.lpDot(x , DiscountedBalance))) 
    print('Now starting with iteration number:', no_iter)
    print('amount of constraints added in this iteration', 
          AdditionalConstraints['TopX'].count())
else:
    print('total number of solving iterations performed:', no_iter)
    if InitialPoolCut == False: 
            print("ADB of contracts repurchased = ", 
                  PreviousPool.sum() - pulp.value(pulp.lpDot(x , PreviousPool)))
    df = retrieve_sol()
    frames = [EBB, df]
    AdditionalConstraints = CheckBreaches()
    if AdditionalConstraints['TopX'].count()!=0:
        print('solution is not optimal, number of clients in breach are:', 
              AdditionalConstraints['TopX'].count())

Status: Optimal
Lapda =  12.839997828006744
Now starting with iteration number: 1
amount of constraints added in this iteration 9
total number of solving iterations needed: 1


This is the heart of the Lazy Constraint program written. Constraints are added for the breaches that are encountered, after which the problem is solved again and subsequently check for breaches until no further breaches are outstanding. The number of iterations in this while loop is capped to make sure that we don't end up in an infinite loop.

In [11]:
#print final solution to a csv file
df = retrieve_sol()
frames = [EBB, df]
EBB_sol = pd.concat(frames, axis=1)
EBB_sol.to_csv('final_sol.csv', index = False, header=True)

#calculate runtime
end_time = time.time()
runtime = (end_time - start_time)/60
print('this program ran for', runtime, 'minutes')

#prints constraints to excel file
with pd.ExcelWriter('diagnostics.xlsx') as writer:  
   TotalConstraints.to_excel(writer, sheet_name='Constraints')
   AdditionalConstraints.to_excel(writer, sheet_name='Breaches')

this program ran for 4.255829139550527 minutes


the final solution is written to a csv file (along with the relevant diagnostics, which is written to a separate file)